In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np

import json

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/r3_inframind/datasets/final_raw_material_quality_prediction.csv')

In [ ]:
df.head()

,Observation,Y-Kappa,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
0,31-00:00,23.10,16.520,121.717,1177.607,169.805,358.282,329.545,1.443,599.253,NaN,16.471,46.011,67.122,329.432,303.099,175.964,1127.197,1319.039,257.325,54.612,252.077,NaN
1,31-01:00,27.60,16.810,79.022,1328.360,341.327,351.050,329.067,1.549,537.201,6.076,16.543,43.954,60.012,330.823,304.879,163.202,665.975,1297.317,241.182,46.603,251.406,29.11
2,31-02:00,23.19,16.709,79.562,1329.407,239.161,350.022,329.260,1.600,549.611,NaN,16.559,44.495,61.304,329.140,303.383,164.013,677.534,1327.072,237.272,51.795,251.335,NaN
3,31-03:00,23.60,16.478,81.011,1334.877,213.527,350.938,331.142,1.604,623.362,6.054,16.562,45.592,68.496,328.875,302.254,181.487,767.853,1324.461,239.478,54.846,250.312,29.02
4,31-04:00,22.90,15.618,93.244,1334.168,243.131,351.640,332.709,NaN,638.672,6.110,16.677,45.512,70.022,328.352,300.954,183.929,888.448,1343.424,215.372,54.186,249.916,29.01


In [ ]:
df.columns

Index(['Observation', 'Y-Kappa', 'ChipRate', 'BF-CMratio', 'BlowFlow',
       'ChipLevel4 ', 'T-upperExt-2 ', 'T-lowerExt-2  ', 'UCZAA',
       'WhiteFlow-4 ', 'AAWhiteSt-4 ', 'AA-Wood-4  ', 'ChipMoisture-4 ',
       'SteamFlow-4 ', 'Lower-HeatT-3', 'Upper-HeatT-3 ', 'ChipMass-4 ',
       'WeakLiquorF ', 'BlackFlow-2 ', 'WeakWashF ', 'SteamHeatF-3 ',
       'T-Top-Chips-4 ', 'SulphidityL-4 '],
      dtype='object')

In [ ]:
  df.drop(['Observation'], axis = 1, inplace = True) 

In [ ]:
df = df.dropna()

In [ ]:
df.head()

,Y-Kappa,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
1,27.60,16.810,79.022,1328.360,341.327,351.050,329.067,1.549,537.201,6.076,16.543,43.954,60.012,330.823,304.879,163.202,665.975,1297.317,241.182,46.603,251.406,29.110
3,23.60,16.478,81.011,1334.877,213.527,350.938,331.142,1.604,623.362,6.054,16.562,45.592,68.496,328.875,302.254,181.487,767.853,1324.461,239.478,54.846,250.312,29.020
6,22.65,14.100,91.887,1307.852,288.989,352.321,331.162,1.468,625.549,6.143,16.620,45.797,71.298,329.662,301.539,179.886,837.178,1315.111,234.047,53.805,249.971,29.220
8,24.70,13.850,96.208,1334.892,362.511,352.372,327.358,1.515,553.172,6.199,16.625,45.541,64.249,332.264,305.419,166.120,909.810,1318.725,180.375,48.842,251.121,29.210
10,24.40,14.117,85.998,1330.104,394.234,348.089,319.027,1.429,540.558,6.209,16.661,45.258,62.179,329.831,302.652,163.258,827.107,1312.372,132.163,49.446,251.406,29.292


In [ ]:
df.head()

,Y-Kappa,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
1,27.60,16.810,79.022,1328.360,341.327,351.050,329.067,1.549,537.201,6.076,16.543,43.954,60.012,330.823,304.879,163.202,665.975,1297.317,241.182,46.603,251.406,29.110
3,23.60,16.478,81.011,1334.877,213.527,350.938,331.142,1.604,623.362,6.054,16.562,45.592,68.496,328.875,302.254,181.487,767.853,1324.461,239.478,54.846,250.312,29.020
6,22.65,14.100,91.887,1307.852,288.989,352.321,331.162,1.468,625.549,6.143,16.620,45.797,71.298,329.662,301.539,179.886,837.178,1315.111,234.047,53.805,249.971,29.220
8,24.70,13.850,96.208,1334.892,362.511,352.372,327.358,1.515,553.172,6.199,16.625,45.541,64.249,332.264,305.419,166.120,909.810,1318.725,180.375,48.842,251.121,29.210
10,24.40,14.117,85.998,1330.104,394.234,348.089,319.027,1.429,540.558,6.209,16.661,45.258,62.179,329.831,302.652,163.258,827.107,1312.372,132.163,49.446,251.406,29.292


In [ ]:
df.describe()

,Y-Kappa,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
count,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000
mean,20.665725,14.395328,86.793969,1240.197595,265.495664,356.834206,324.133267,1.497626,595.079710,6.146672,17.896412,46.826756,67.180282,325.265771,300.264160,162.279008,865.121061,1163.291947,273.059489,49.468351,251.266290,30.497626
std,3.021990,1.496178,7.533255,72.375493,81.847426,9.403771,7.847815,0.106500,68.208005,0.081359,0.891370,1.443805,5.548357,4.588467,4.578489,14.353163,126.237074,150.955336,158.311244,4.706549,1.276348,0.657965
min,12.170000,10.833000,68.645000,954.092000,0.000000,340.222000,287.705000,1.182000,405.111000,5.890000,16.543000,43.395000,51.192000,318.821000,293.312000,113.922000,493.856000,838.948000,0.000000,35.510000,248.390000,29.020000
25%,18.490000,13.481000,81.164500,1198.625500,220.354000,350.338000,321.453000,1.437000,542.125000,6.096000,17.136500,45.777500,63.780500,321.282500,296.566500,153.032500,769.505500,1017.102500,151.810000,46.520000,250.307000,30.155000
50%,20.700000,14.417000,86.950000,1272.030000,274.516000,357.520000,325.690000,1.500000,596.506000,6.140000,17.798000,46.906000,68.023000,322.913000,298.517000,163.067000,860.372000,1141.946000,284.877000,49.987000,251.406000,30.400000
75%,23.185000,15.529500,92.024500,1289.225000,332.497000,362.392500,329.433500,1.566000,652.506500,6.200000,18.595500,48.006000,71.791500,329.821000,304.089500,173.134000,967.372000,1313.741500,396.471500,52.683000,252.423000,30.920000
max,27.600000,16.817000,114.570000,1334.892000,419.014000,399.135000,333.706000,1.747000,731.394000,6.340000,19.581000,50.658000,76.147000,333.596000,310.487000,187.665000,1132.181000,1392.868000,598.171000,63.332000,254.122000,32.840000


In [ ]:
df.columns

Index(['Y-Kappa', 'ChipRate', 'BF-CMratio', 'BlowFlow', 'ChipLevel4 ',
       'T-upperExt-2 ', 'T-lowerExt-2  ', 'UCZAA', 'WhiteFlow-4 ',
       'AAWhiteSt-4 ', 'AA-Wood-4  ', 'ChipMoisture-4 ', 'SteamFlow-4 ',
       'Lower-HeatT-3', 'Upper-HeatT-3 ', 'ChipMass-4 ', 'WeakLiquorF ',
       'BlackFlow-2 ', 'WeakWashF ', 'SteamHeatF-3 ', 'T-Top-Chips-4 ',
       'SulphidityL-4 '],
      dtype='object')

In [ ]:
# Model training

In [ ]:
is_defected = []
for i in df.index:
  if(df['Y-Kappa'][i]<18):
    is_defected.append(1)
  else:
    is_defected.append(0)
df['is_defected'] = is_defected
df = df.drop('Y-Kappa', axis = "columns") 

In [ ]:
# Randomize rows
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
len(df.index)

131

In [ ]:
df.head()

,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4,is_defected
0,15.225,82.030,1280.837,312.879,358.944,327.582,1.335,615.580,6.147,18.712,48.818,72.743,321.206,296.893,164.746,965.018,1043.596,213.414,49.987,251.192,30.595,0
1,12.483,94.919,1184.907,284.859,360.734,315.120,1.513,520.670,6.093,17.168,45.304,61.695,329.950,303.678,144.930,973.795,1345.998,439.382,44.248,252.804,30.930,0
2,13.917,87.835,1190.889,390.140,341.081,320.888,1.346,492.543,6.058,18.354,43.830,54.213,319.974,295.580,136.052,663.228,995.420,47.068,35.510,252.662,30.210,0
3,12.342,91.188,1186.963,243.359,361.581,325.371,1.447,611.877,6.110,19.092,48.324,66.382,323.430,295.814,157.677,661.041,1118.696,259.029,50.812,249.298,30.450,0
4,13.058,90.864,1186.527,225.757,360.327,316.188,1.503,517.272,6.050,17.061,45.370,60.040,329.851,303.939,146.673,898.722,1344.346,439.608,46.053,252.985,30.820,0


In [ ]:
# train test split by specifying row number to split at
def train_test_split(data, pred_feature_name, row_to_split_at):  
  y = data[pred_feature_name].copy()
  x = data.drop(pred_feature_name, axis=1).copy()
  
  y_train = y.iloc[:row_to_split_at] #indexes rows for test data
  X_train = x.iloc[:row_to_split_at] #indexes rows for training data

  X_test = x.iloc[row_to_split_at:] #indexes rows for training data
  y_test = y.iloc[row_to_split_at:] #indexes rows for test data

  return X_train, X_test, y_train, y_test

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(df,'is_defected', 100)

In [ ]:
X_train

,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
0,15.225,82.030,1280.837,312.879,358.944,327.582,1.335,615.580,6.147,18.712,48.818,72.743,321.206,296.893,164.746,965.018,1043.596,213.414,49.987,251.192,30.595
1,12.483,94.919,1184.907,284.859,360.734,315.120,1.513,520.670,6.093,17.168,45.304,61.695,329.950,303.678,144.930,973.795,1345.998,439.382,44.248,252.804,30.930
2,13.917,87.835,1190.889,390.140,341.081,320.888,1.346,492.543,6.058,18.354,43.830,54.213,319.974,295.580,136.052,663.228,995.420,47.068,35.510,252.662,30.210
3,12.342,91.188,1186.963,243.359,361.581,325.371,1.447,611.877,6.110,19.092,48.324,66.382,323.430,295.814,157.677,661.041,1118.696,259.029,50.812,249.298,30.450
4,13.058,90.864,1186.527,225.757,360.327,316.188,1.503,517.272,6.050,17.061,45.370,60.040,329.851,303.939,146.673,898.722,1344.346,439.608,46.053,252.985,30.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.033,86.998,1284.668,280.871,364.976,329.189,1.400,633.229,6.194,18.200,46.764,71.008,321.979,298.587,167.277,859.354,995.551,194.496,50.608,248.751,31.490
96,13.250,100.322,1301.675,241.529,359.910,328.609,1.418,690.196,6.150,18.597,48.848,73.780,321.330,297.975,156.148,863.677,918.058,0.000,50.869,252.143,30.540
97,15.542,82.392,1280.501,334.242,348.771,316.870,1.596,558.715,6.220,17.261,46.713,68.318,331.510,309.023,156.631,859.558,1367.960,300.409,49.837,252.626,30.480
98,16.810,79.022,1328.360,341.327,351.050,329.067,1.549,537.201,6.076,16.543,43.954,60.012,330.823,304.879,163.202,665.975,1297.317,241.182,46.603,251.406,29.110


In [ ]:
X_test

,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
100,10.833,93.121,1202.818,339.119,342.535,324.194,1.258,601.694,6.049,18.447,45.057,63.969,322.040,299.894,146.584,602.040,838.948,48.632,41.508,251.630,30.650
101,13.242,81.087,1253.469,299.795,369.828,332.648,1.662,563.039,6.135,17.109,46.068,66.548,331.427,305.347,164.670,663.686,1293.743,375.268,52.562,251.823,29.850
102,16.325,75.644,1234.263,158.626,358.079,330.166,1.650,700.063,6.130,19.289,48.317,73.429,320.512,293.925,177.287,863.677,1093.523,445.041,58.004,250.226,30.360
103,14.017,84.943,1189.913,224.176,341.610,321.956,1.434,586.588,6.054,18.396,45.715,62.809,318.821,296.005,162.546,741.200,996.419,0.000,47.162,252.001,29.970
104,14.517,91.178,1243.059,169.592,359.808,332.637,1.588,731.394,6.135,19.288,48.454,74.091,321.767,293.451,183.506,746.032,1147.552,38.317,59.564,249.763,29.950
105,14.375,83.036,1193.646,111.753,367.377,333.370,1.710,642.588,5.890,17.979,48.222,72.265,326.828,299.157,170.520,746.643,1246.745,486.586,52.135,251.594,30.290
106,13.242,94.924,1215.824,251.125,345.485,308.905,1.517,520.162,6.099,17.201,46.032,60.398,329.865,304.102,147.906,972.167,1362.245,219.321,45.439,252.835,31.220
107,14.533,89.558,1301.573,267.787,369.879,327.704,1.623,593.811,6.130,17.026,47.625,73.155,331.063,305.413,174.069,931.782,1319.036,375.503,54.777,251.212,30.130
108,15.467,79.303,1228.541,204.661,352.321,323.737,1.428,670.156,6.195,17.754,46.303,70.678,320.266,297.618,182.769,833.720,1122.170,414.797,53.493,250.424,29.910
109,12.875,92.564,1191.757,229.014,347.377,326.687,1.552,521.840,6.000,16.822,45.969,60.485,328.410,303.215,152.918,763.072,1293.557,291.099,49.835,252.072,29.530


In [ ]:
# # FOR CLASSIFICATION PROBLEM,
# # IF dataset is imbalanced
# from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import RandomOverSampler
# smote = RandomOverSampler( sampling_strategy=0.5)
# # 0.5 means minority = majority/2
# X_train, y_train = smote.fit_sample(X_train, y_train)

In [ ]:
X_train

,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
0,15.225,82.030,1280.837,312.879,358.944,327.582,1.335,615.580,6.147,18.712,48.818,72.743,321.206,296.893,164.746,965.018,1043.596,213.414,49.987,251.192,30.595
1,12.483,94.919,1184.907,284.859,360.734,315.120,1.513,520.670,6.093,17.168,45.304,61.695,329.950,303.678,144.930,973.795,1345.998,439.382,44.248,252.804,30.930
2,13.917,87.835,1190.889,390.140,341.081,320.888,1.346,492.543,6.058,18.354,43.830,54.213,319.974,295.580,136.052,663.228,995.420,47.068,35.510,252.662,30.210
3,12.342,91.188,1186.963,243.359,361.581,325.371,1.447,611.877,6.110,19.092,48.324,66.382,323.430,295.814,157.677,661.041,1118.696,259.029,50.812,249.298,30.450
4,13.058,90.864,1186.527,225.757,360.327,316.188,1.503,517.272,6.050,17.061,45.370,60.040,329.851,303.939,146.673,898.722,1344.346,439.608,46.053,252.985,30.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.033,86.998,1284.668,280.871,364.976,329.189,1.400,633.229,6.194,18.200,46.764,71.008,321.979,298.587,167.277,859.354,995.551,194.496,50.608,248.751,31.490
96,13.250,100.322,1301.675,241.529,359.910,328.609,1.418,690.196,6.150,18.597,48.848,73.780,321.330,297.975,156.148,863.677,918.058,0.000,50.869,252.143,30.540
97,15.542,82.392,1280.501,334.242,348.771,316.870,1.596,558.715,6.220,17.261,46.713,68.318,331.510,309.023,156.631,859.558,1367.960,300.409,49.837,252.626,30.480
98,16.810,79.022,1328.360,341.327,351.050,329.067,1.549,537.201,6.076,16.543,43.954,60.012,330.823,304.879,163.202,665.975,1297.317,241.182,46.603,251.406,29.110


In [ ]:
X_test

,ChipRate,BF-CMratio,BlowFlow,ChipLevel4,T-upperExt-2,T-lowerExt-2,UCZAA,WhiteFlow-4,AAWhiteSt-4,AA-Wood-4,ChipMoisture-4,SteamFlow-4,Lower-HeatT-3,Upper-HeatT-3,ChipMass-4,WeakLiquorF,BlackFlow-2,WeakWashF,SteamHeatF-3,T-Top-Chips-4,SulphidityL-4
100,10.833,93.121,1202.818,339.119,342.535,324.194,1.258,601.694,6.049,18.447,45.057,63.969,322.040,299.894,146.584,602.040,838.948,48.632,41.508,251.630,30.650
101,13.242,81.087,1253.469,299.795,369.828,332.648,1.662,563.039,6.135,17.109,46.068,66.548,331.427,305.347,164.670,663.686,1293.743,375.268,52.562,251.823,29.850
102,16.325,75.644,1234.263,158.626,358.079,330.166,1.650,700.063,6.130,19.289,48.317,73.429,320.512,293.925,177.287,863.677,1093.523,445.041,58.004,250.226,30.360
103,14.017,84.943,1189.913,224.176,341.610,321.956,1.434,586.588,6.054,18.396,45.715,62.809,318.821,296.005,162.546,741.200,996.419,0.000,47.162,252.001,29.970
104,14.517,91.178,1243.059,169.592,359.808,332.637,1.588,731.394,6.135,19.288,48.454,74.091,321.767,293.451,183.506,746.032,1147.552,38.317,59.564,249.763,29.950
105,14.375,83.036,1193.646,111.753,367.377,333.370,1.710,642.588,5.890,17.979,48.222,72.265,326.828,299.157,170.520,746.643,1246.745,486.586,52.135,251.594,30.290
106,13.242,94.924,1215.824,251.125,345.485,308.905,1.517,520.162,6.099,17.201,46.032,60.398,329.865,304.102,147.906,972.167,1362.245,219.321,45.439,252.835,31.220
107,14.533,89.558,1301.573,267.787,369.879,327.704,1.623,593.811,6.130,17.026,47.625,73.155,331.063,305.413,174.069,931.782,1319.036,375.503,54.777,251.212,30.130
108,15.467,79.303,1228.541,204.661,352.321,323.737,1.428,670.156,6.195,17.754,46.303,70.678,320.266,297.618,182.769,833.720,1122.170,414.797,53.493,250.424,29.910
109,12.875,92.564,1191.757,229.014,347.377,326.687,1.552,521.840,6.000,16.822,45.969,60.485,328.410,303.215,152.918,763.072,1293.557,291.099,49.835,252.072,29.530


In [ ]:
type(X_train), type(X_test), type(y_train) ,type(y_test)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [ ]:
# import evaluation metrices
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report
from sklearn.svm import SVC



In [ ]:
clf = ExtraTreesClassifier(random_state=0,n_estimators=100)
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)
print('ExtraTreesClassifier accuracy: {}'.format(accuracy_score(y_test, y_pred)))
print(f'The accuracy score: {accuracy_score(y_test, y_pred)}')
print(f'The F1 Score is: {f1_score(y_test,y_pred)}')
print(classification_report(y_test, y_pred))


ExtraTreesClassifier accuracy: 0.8064516129032258
The accuracy score: 0.8064516129032258
The F1 Score is: 0.6250000000000001
              precision    recall  f1-score   support

           0       0.91      0.83      0.87        24
           1       0.56      0.71      0.63         7

    accuracy                           0.81        31
   macro avg       0.73      0.77      0.75        31
weighted avg       0.83      0.81      0.81        31



In [ ]:
clf3 = AdaBoostClassifier(n_estimators=100)
clf3.fit(X_train, y_train)
y_pred= clf3.predict(X_test)
print('AdaBoostClassifier accuracy: {}'.format(accuracy_score(y_test, y_pred)))
print(f'The accuracy score: {accuracy_score(y_test, y_pred)}')
print(f'The F1 Score is: {f1_score(y_test,y_pred)}')
print(classification_report(y_test, y_pred))



AdaBoostClassifier accuracy: 0.7419354838709677
The accuracy score: 0.7419354838709677
The F1 Score is: 0.5
              precision    recall  f1-score   support

           0       0.86      0.79      0.83        24
           1       0.44      0.57      0.50         7

    accuracy                           0.74        31
   macro avg       0.65      0.68      0.66        31
weighted avg       0.77      0.74      0.75        31



In [ ]:

clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)
print('GradientBoostingClassifier accuracy: {}'.format(accuracy_score(y_test, y_pred)))
print(f'The accuracy score: {accuracy_score(y_test, y_pred)}')
print(f'The F1 Score is: {f1_score(y_test,y_pred)}')
print(classification_report(y_test, y_pred))



GradientBoostingClassifier accuracy: 0.8064516129032258
The accuracy score: 0.8064516129032258
The F1 Score is: 0.6666666666666665
              precision    recall  f1-score   support

           0       0.95      0.79      0.86        24
           1       0.55      0.86      0.67         7

    accuracy                           0.81        31
   macro avg       0.75      0.82      0.77        31
weighted avg       0.86      0.81      0.82        31



In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

train_pred = logreg.predict(X_train)
logreg_pred = logreg.predict(X_test)

print('Accuracy of Logistic regression classifier on TRAINING set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print(classification_report(y_train, train_pred))

print('Accuracy of Logistic regression classifier on TEST set: {:.2f}'
     .format(logreg.score(X_test, y_test)))
print(classification_report(y_test, logreg_pred))

Accuracy of Logistic regression classifier on TRAINING set: 0.89
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        79
           1       0.78      0.67      0.72        21

    accuracy                           0.89       100
   macro avg       0.85      0.81      0.82       100
weighted avg       0.89      0.89      0.89       100

Accuracy of Logistic regression classifier on TEST set: 0.74
              precision    recall  f1-score   support

           0       0.81      0.88      0.84        24
           1       0.40      0.29      0.33         7

    accuracy                           0.74        31
   macro avg       0.60      0.58      0.59        31
weighted avg       0.72      0.74      0.73        31



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
print(y_test.value_counts())
# print(type(logreg_pred))
print(pd.Series(logreg_pred).value_counts())

0    24
1     7
Name: is_defected, dtype: int64
0    26
1     5
dtype: int64


In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier().fit(X_train, y_train)

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

clf_pred = clf.predict(X_test)
print(classification_report(y_test, clf_pred))

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 0.77
              precision    recall  f1-score   support

           0       0.90      0.79      0.84        24
           1       0.50      0.71      0.59         7

    accuracy                           0.77        31
   macro avg       0.70      0.75      0.72        31
weighted avg       0.81      0.77      0.79        31



In [ ]:
clf2 = DecisionTreeClassifier(max_depth=12).fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf2.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf2.score(X_test, y_test)))

clf_pred2 = (clf2.predict_proba(X_test)[:,1] >= 0.5).astype(bool) # set threshold as 0.3
# clf_pred2 = clf2.predict(X_test)
print(classification_report(y_test, clf_pred2))

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 0.68
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        24
           1       0.38      0.71      0.50         7

    accuracy                           0.68        31
   macro avg       0.64      0.69      0.63        31
weighted avg       0.78      0.68      0.70        31



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=0)
rf.fit(X_train, y_train)
# TRAIN SET EVAL
print('Accuracy of Random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))
print(classification_report(y_train, train_pred))

# TEST SET EVAL
print('Accuracy of Random forest classifier on test set: {:.2f}'
     .format(rf.score(X_test, y_test)))

# rf_pred = rf.predict(X_test)
rf_pred = (rf.predict_proba(X_test)[:,1] >= 0.5).astype(bool) # set threshold as 0.3
train_pred = rf.predict(X_train)

print(classification_report(y_test, rf_pred))

Accuracy of Random forest classifier on training set: 1.00
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        79
           1       0.78      0.67      0.72        21

    accuracy                           0.89       100
   macro avg       0.85      0.81      0.82       100
weighted avg       0.89      0.89      0.89       100

Accuracy of Random forest classifier on test set: 0.74
              precision    recall  f1-score   support

           0       0.86      0.79      0.83        24
           1       0.44      0.57      0.50         7

    accuracy                           0.74        31
   macro avg       0.65      0.68      0.66        31
weighted avg       0.77      0.74      0.75        31



In [ ]:
# SAVE THE MODEL
import pickle
pkl_filename = '/content/drive/MyDrive/r3_inframind/models/defect_detection.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(rf, file)